# Post Training Quantization a Yolo8-nano Object Detection Model

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/example_keras_nanodet_plus.ipynb)

## Overview


In this tutorial, we'll demonstrate the post-training quantization using MCT for a pre-trained object detection model in Keras. Specifically, we'll integrate post-processing, including the non-maximum suppression (NMS) layer, into the model. This integration aligns with the imx500 target platform capabilities.

In this example we will use an existing pre-trained Yolo8-nano model taken from [https://github.com/ultralytics/ultralytics](https://github.com/ultralytics/ultralytics). We will convert the model to a Tensorflow model that includes box decoding and NMS layer. Further, we will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on COCO dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of Keras object detection model including the post-processing.
2. Data preparation - loading and preprocessing validation and representative datasets from COCO.
3. Accuracy evaluation of the floating-point and the quantized models.

## Setup
Install the relevant packages.

In [ ]:
!pip install -q torch
!pip install -q tensorflow
!pip install -q pycocotools
!pip install -q model-compression-toolkit

 Clone a copy of the [MCT](https://github.com/sony/model_optimization) (Model Compression Toolkit) into your current directory. This step ensures that you have access to the tutorials [resources](https://github.com/sony/model_optimization/tree/main/tutorials/resources) folder which contains all the necessary utility functions for this tutorial

In [ ]:
!git clone https://github.com/sony/model_optimization.git local_mct
import sys
sys.path.insert(0,"/content/local_mct")

Load COCO evaluation set

In [ ]:
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q -o annotations_trainval2017.zip -d /content/coco
!echo Done loading annotations
!wget -nc http://images.cocodataset.org/zips/val2017.zip
!unzip -q -o val2017.zip -d /content/coco
!echo Done loading val2017 images

Lastly, download the pre-trained weights of `YOLOv8n` from [Ultralytics](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt) 

In [ ]:
!wget -nc https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

## Floating Point Model

### Load the pre-trained weights of Yolo8-nano
We begin by loading the pre-trained weights of `YOLOv8n` using `torch.load`, as the original model is in PyTorch format. Please make sure the pre-trained weights are located in the `content` directory or specify the correct path.

In [ ]:
import torch

PRETRAINED_WEIGHTS_FILE = '/content/yolov8n.pt'
pretrained_weights = torch.load(PRETRAINED_WEIGHTS_FILE, map_location=torch.device('cpu'))

### Generate Yolo8-nano Keras model
In the following steps, we proceed to create a Keras model based on [Ultralytics](https://github.com/ultralytics/ultralytics). We then initialize the model weights to match those of the original model.

To this base model, we integrate the post-processing components, which include box decoding layers following by tensorflow [tf.image.combined_non_max_suppression](https://www.tensorflow.org/api_docs/python/tf/image/combined_non_max_suppression) layer.
For further insights into the model's implementation details, please refer to the tutorials ["resources" folder](https://github.com/sony/model_optimization/tree/main/tutorials/resources).   

In [ ]:
import tensorflow as tf
from keras.models import Model
from tutorials.resources.utils.torch2keras_weights_translation import load_state_dict
from tutorials.resources.yolov8.yolov8_keras import yolov8_keras

# Parameters of nanodet-plus-m-1.5x_416
INPUT_RESOLUTION = 640

# Generate Yolov8n model 
model = yolov8_keras('/content/local_mct/model_optimization/tutorials/resources/yolov8/yolov8n.yaml', INPUT_RESOLUTION)

# Set the pre-trained weights
load_state_dict(model, state_dict_torch=pretrained_weights)

# Add Tensorflow NMS layer
boxes, scores  = model.output
outputs = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=300,
    max_total_size=300,
    iou_threshold=0.65,
    score_threshold=0.001,
    pad_per_class=False,
    clip_boxes=False
    )

model = Model(model.input, outputs, name='yolov8n')

print('Model is ready for evaluation')

#### Evaluate the floating point model
Next, we evaluate the floating point model by using `cocoeval` library alongside additional dataset utilities. We can verify the mAP accuracy aligns with that of the original model. 
Note that we set the "batch_size" to 5 and the preprocessing according to [Nanodet](https://github.com/RangiLyu/nanodet/tree/main).
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
import cv2
import numpy as np
from tutorials.resources.coco_evaluation import coco_dataset_generator, CocoEval
from tutorials.resources.yolov8.yolov8_keras import yolov8_preprocess

EVAL_DATASET_FOLDER = '/content/coco/images/val2017'
EVAL_DATASET_ANNOTATION_FILE = '/content/coco/annotations/instances_val2017.json'
BATCH_SIZE = 5

# Load COCO evaluation set
val_dataset = coco_dataset_generator(dataset_folder=EVAL_DATASET_FOLDER,
                                     annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess,
                                     batch_size=BATCH_SIZE)

# Define resizing information to map between the model's output and the original image dimensions
output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': True}

# Initialize the evaluation metric object
coco_metric = CocoEval(EVAL_DATASET_ANNOTATION_FILE, output_resize) 

# Iterate and the evaluation set
for batch_idx, (images, targets) in enumerate(val_dataset):
    
    # Run inference on the batch
    outputs = model(images)

    # Add the model outputs to metric object (a dictionary of outputs after postprocess: boxes, scores & classes)
    coco_metric.add_batch_detections(outputs, targets)
    if (batch_idx + 1) % 100 == 0:
        print(f'processed {(batch_idx + 1) * BATCH_SIZE} images')

# Print float model mAP results
print("Float model mAP: {:.4f}".format(coco_metric.result()[0]))

## Quantize Model

### Post training quantization using Model Compression Toolkit 

Now we are ready to use MCT's post training quantization! We will define a representative dataset based on the training dataset and preform the model quantization. We will use 100 representative images for calibration (20 iterations of "batch_size" images each).
Same as the above section, please ensure that the dataset path has been set correctly.

In [ ]:
import model_compression_toolkit as mct

TRAIN_DATASET_FOLDER = '/content/coco/images/val2017/'
TRAIN_DATASET_ANNOTATION_FILE = '/content/coco/annotations/instances_val2017.json'
n_iters = 20

# Load COCO train set
train_dataset = coco_dataset_generator(dataset_folder=TRAIN_DATASET_FOLDER,
                                       annotation_file=TRAIN_DATASET_ANNOTATION_FILE,
                                       preprocess=yolov8_preprocess,
                                       batch_size=BATCH_SIZE)

# Define representative dataset generator
def get_representative_dataset(n_iter, train_loader):

    def representative_dataset():
        ds_iter = iter(train_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Preform post training quantization 
quant_model, _ = mct.ptq.keras_post_training_quantization_experimental(model,
                                                                       get_representative_dataset(n_iters, train_dataset))

print('Quantized model is ready')

### Evaluate quantized model
Lastly, we can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as EPTQ (Enhanced Post Training Quantization).

In [ ]:
# Re-load COCO evaluation set
val_dataset = coco_dataset_generator(dataset_folder=EVAL_DATASET_FOLDER,
                                     annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess,
                                     batch_size=BATCH_SIZE)

# Initialize the evaluation metric object
coco_metric = CocoEval(EVAL_DATASET_ANNOTATION_FILE, output_resize) 

# Iterate and the evaluation set
for batch_idx, (images, targets) in enumerate(val_dataset):
    # Run inference on the batch
    outputs = quant_model(images)

    # Add the model outputs to metric object (a dictionary of outputs after postprocess: boxes, scores & classes)
    coco_metric.add_batch_detections(outputs, targets)
    if (batch_idx + 1) % 100 == 0:
        print(f'processed {(batch_idx + 1) * BATCH_SIZE} images')

# Print quantized model mAP results
print("Quantized model mAP: {:.4f}".format(coco_metric.result()[0]))